<a href="https://colab.research.google.com/github/gbr-guimaraes/Projeto-organizador-de-estudos/blob/main/Projeto_Cronograma_de_estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das bibliotecas necessárias


In [1]:
!pip install -q -U google-generativeai
!pip install PyPDF2 -q

# Importação das bibliotecas

In [2]:
from google.colab import userdata
import google.generativeai as genai
import os
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, Markdown
import textwrap

# Configurações

In [3]:
API_KEY = userdata.get('SECRET_KEY')  # Obtém a chave de API do Google Gemini
max_token = 1048576                    # Define o limite máximo de tokens para o modelo
list_command = "Apresente os cargos disponiveis neste edital em formato de lista simples,apenas texto, um elemento em cada linha, sem detalhes, textos de introdução ou caracteres especiais"  # Comando para extrair a lista de cargos do edital
main_command = "A partir do texto acima, crie um cronograma de estudos personalizado, em formato de tabela Markdown, com os conteúdos gerais e específicos para o cargo de {role} para o estudante {student_name}, este aluno tem disponibilidade de estudar {hours} horas por dia, nos dias {week_days}"  # Comando para gerar o cronograma de estudos

# Configuração do modelo

In [4]:
genai.configure(api_key=API_KEY)          # Configura a API com a chave obtida
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Define o modelo de linguagem a ser usado (Gemini Pro)
convo = model.start_chat(history=[])     # Inicia uma conversa com o modelo, mantendo o histórico vazio

# Função para extrair texto do PDF

In [5]:
def extract_pdf_text(file_path: str) -> str:
    """Extrai o texto de um arquivo PDF."""
    with open(file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "".join(page.extract_text() for page in pdf_reader.pages)  # Extrai o texto de cada página e junta em uma única string

# Função para obter a lista de cargos

In [6]:
def get_role_list(pdf_text: str) -> list[str]:
    """Obtém a lista de cargos disponíveis no edital."""
    convo.send_message(pdf_text + list_command)  # Envia o texto do edital juntamente com o comando para extrair a lista de cargos
    return convo.last.text.splitlines()  # Retorna a lista de cargos, separando cada cargo por linha

# Função para gerar o cronograma

In [7]:
def generate_study_schedule(pdf_text: str, role: str, student_name: str, week_days: str, hours: int):
    """Gera um cronograma de estudos personalizado."""
    convo.send_message(pdf_text + "\n\n" + main_command.format(role=role, student_name=student_name, hours=hours, week_days=week_days))  # Envia o texto do edital e o comando para gerar o cronograma, formatando o comando com os dados fornecidos
    display(to_markdown(convo.last.text))  # Exibe o cronograma gerado pelo modelo, formatado em Markdown

# Função para formatar a resposta da IA

In [8]:
def to_markdown(text):
    """Formata o texto para Markdown."""
    text = text.replace('•', '  *')  # Substitui marcadores "•" por "*", formatação padrão de Markdown para listas
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))  # Indenta o texto com "> " para melhor visualização em Markdown

# Função para lidar com o upload do arquivo

In [9]:
def handle_file_upload(change):
    """Carrega o arquivo PDF e atualiza a lista de cargos."""
    global uploaded_file_path  # Define a variável uploaded_file_path como global para ser acessada em outras funções
    uploaded_file = change['new']  # Obtém o arquivo carregado pelo usuário
    if uploaded_file:
        uploaded_file_path = "./edital.pdf"  # Define o nome do arquivo como "edital.pdf"
        with open(uploaded_file_path, "wb") as fp:
            fp.write(file_upload.data[0])  # Salva o arquivo carregado em disco
        pdf_text = extract_pdf_text(uploaded_file_path)  # Extrai o texto do arquivo PDF carregado
        roles.options = get_role_list(pdf_text)  # Atualiza a lista de cargos disponíveis no widget dropdown
        submit.disabled = False  # Habilita o botão "Gerar" após o arquivo ser carregado
        print("Arquivo carregado com sucesso!")

# Função para processar o clique no botão "Gerar"

In [10]:
def on_button_clicked(b):
    """Gera o cronograma com as informações fornecidas."""
    if uploaded_file_path:
        checkboxes = [monday, tuesday, wednesday, thursday, friday, saturday, sunday]  # Cria uma lista com os widgets checkbox dos dias da semana
        week_list = ", ".join([checkbox.description for checkbox in checkboxes if checkbox.value])  # Cria uma string com os dias da semana selecionados
        pdf_text = extract_pdf_text(uploaded_file_path)  # Extrai o texto do arquivo PDF carregado
        generate_study_schedule(pdf_text, roles.value, name.value, week_list, time_avaliable.value)  # Chama a função para gerar o cronograma, passando as informações coletadas

# Criação dos widgets da interface

In [11]:
file_upload = widgets.FileUpload(accept='.pdf',multiple=False)  # Widget para carregar o arquivo PDF
monday = widgets.Checkbox(value=False, description='Segunda-Feira')  # Checkbox para selecionar segunda-feira
tuesday = widgets.Checkbox(value=False, description='Terça-Feira')  # Checkbox para selecionar terça-feira
wednesday = widgets.Checkbox(value=False, description='Quarta-Feira')  # Checkbox para selecionar quarta-feira
thursday = widgets.Checkbox(value=False, description='Quinta-Feira')  # Checkbox para selecionar quinta-feira
friday = widgets.Checkbox(value=False, description='Sexta-Feira')  # Checkbox para selecionar sexta-feira
saturday = widgets.Checkbox(value=False, description='Sábado')  # Checkbox para selecionar sábado
sunday = widgets.Checkbox(value=False, description='Domingo')  # Checkbox para selecionar domingo
time_avaliable = widgets.BoundedIntText(value=5, min=1, max=24, step=1)  # Campo numérico para definir a quantidade de horas disponíveis por dia
name = widgets.Text(value='', placeholder='Nome...')  # Campo de texto para inserir o nome do usuário
roles = widgets.Dropdown(options=["Selecione um cargo"], value="Selecione um cargo")  # Dropdown para selecionar o cargo desejado (será preenchido após o carregamento do PDF)
submit = widgets.Button(description='Gerar', disabled=True)  # Botão para gerar o cronograma, inicialmente desabilitado

# Exibição dos widgets e eventos

In [12]:
file_upload_box = widgets.VBox([widgets.Label("Faça o upload do seu edital:"), file_upload])  # Cria um container vertical para o widget de upload
week_days_box = widgets.VBox([monday, tuesday, wednesday, thursday, friday, saturday, sunday])  # Cria um container vertical para os checkboxes dos dias da semana
input_box = widgets.VBox([  # Cria um container vertical para os demais widgets
    widgets.Label("Seu Nome:"), name,
    widgets.Label("Cargo desejado:"), roles,
    widgets.Label("Dias disponíveis para estudo:"), week_days_box,
    widgets.Label("Horas/dia:"), time_avaliable,
    submit
])

display(file_upload_box, input_box)  # Exibe os containers na tela

# Eventos
file_upload.observe(handle_file_upload, names='value')  # Associa a função handle_file_upload ao evento de mudança no widget de upload
submit.observe(on_button_clicked)  # Associa a função on_button_clicked ao evento de clique no botão "Gerar"

> ## Cronograma de Estudos - Concurso Público Nacional Unificado 2024 - Estatístico (5 horas/dia)
> 
> **Período:** 11/01/2024 até 04/05/2024 (114 dias)
> 
> **Tempo total de estudo:** 570 horas
> 
> | Dia | Manhã (2.5 horas) | Tarde (2.5 horas) | Revisão (30min - Final de semana) | Simulado (2h - Final de semana) |
> |---|---|---|---|---|
> | Segunda | **Conhecimentos Gerais:** Políticas Públicas (conceitos, ciclos, institucionalização) | **Conhecimentos Específicos:** Gestão Governamental (Planejamento estratégico, BSC, SWOT) |  Políticas Públicas: Conceitos e Ciclos  | Simulado Conhecimentos Gerais - Bloco 1  |
> | Terça | **Conhecimentos Gerais:** Desafios do Estado de Direito (Constituição, Direitos Humanos) | **Conhecimentos Específicos:** Gestão de Projetos (PMBOK, metodologias ágeis) | Gestão Governamental: Planejamento estratégico | Simulado Conhecimentos Específicos - Eixo 1  |
> | Quarta | **Conhecimentos Gerais:** Ética e Integridade (princípios, governança, transparência) | **Conhecimentos Específicos:**  Gestão de Processos (mapeamento, BPMN) | Ética e Integridade: Princípios e Governança | Simulado Conhecimentos Gerais - Bloco 2  |
> | Quinta | **Conhecimentos Gerais:** Diversidade e Inclusão (conceitos, grupos vulnerabilizados) | **Conhecimentos Específicos:** Gestão de Riscos (princípios, processos, boas práticas) | Diversidade e Inclusão: Conceitos e Desafios | Simulado Conhecimentos Específicos - Eixo 2  | 
> | Sexta | **Conhecimentos Gerais:** Administração Pública Federal (princípios, estrutura, regime jurídico) | **Conhecimentos Específicos:** Inovação, Governo Eletrônico, Comunicação, Compras Governamentais | Administração Pública: Princípios e Estrutura | Simulado Conhecimentos Gerais - Bloco 3 |
> | Sábado | **Conhecimentos Gerais:** Finanças Públicas (atribuições, orçamento, federalismo fiscal) | **Conhecimentos Específicos:** Organização da Administração Pública Federal | Finanças Públicas |  |
> | Domingo | DESCANSO | **Conhecimentos Específicos:** Políticas Públicas (elaboração, implementação, avaliação) |  |  |
> 
> **Conteúdo da Semana 2:**
> 
> * **Conhecimentos Gerais:** Revisão da semana anterior +  Leis importantes (PNDH-3, Lei de Responsabilidade Fiscal, etc.)
> * **Conhecimentos Específicos:** Políticas de CT&I, Políticas de Governo Digital, Segurança da Informação
> 
> **Conteúdo da Semana 3:**
> 
> * **Conhecimentos Gerais:** Revisão da semana anterior + Resolução de exercícios
> * **Conhecimentos Específicos:** Governança e Gestão de TI, Sistemas Operacionais, Redes de Computadores
> 
> **Conteúdo da Semana 4:**
> 
> * **Conhecimentos Gerais:** Revisão da semana anterior + Simulados
> * **Conhecimentos Específicos:**  Programação e Estruturas de Dados, Programação Web
> 
> **Conteúdo da Semana 5:**
> 
> * **Conhecimentos Gerais:** Revisão da semana anterior + Revisão geral
> * **Conhecimentos Específicos:** Desenvolvimento para plataformas móveis, Engenharia de Software
> 
> **E assim por diante, adaptando o cronograma conforme o seu ritmo de aprendizado.**
> 
> **Observações:**
> 
> * Este cronograma é apenas uma sugestão, faça as adaptações necessárias de acordo com sua rotina e preferências.
> * Dedique tempo para revisões periódicas e resolução de exercícios.
> * Faça simulados com frequência para avaliar seu progresso e identificar seus pontos fracos.
> * **Concentre-se principalmente no Eixo Temático 5 (Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos) que possui maior peso para o cargo de Estatístico.**
> * Utilize materiais de estudo de qualidade, como livros, apostilas, cursos online e videoaulas.
> * Mantenha-se atualizado sobre as notícias e as mudanças na legislação relacionadas ao concurso.
> * **Boa sorte nos estudos!** 


Arquivo carregado com sucesso!
